# ഓട്ടോജൻ ഉപകരണം ഉപയോഗിക്കുന്ന ഉദാഹരണം


## ആവശ്യമായ പാക്കേജുകൾ ഇറക്കുമതി ചെയ്യുക


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## ക്ലയന്റ് സൃഷ്ടിക്കൽ

ഈ ഉദാഹരണത്തിൽ, LLM-ലേക്ക് ആക്സസ് നേടാൻ [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ഉപയോഗിക്കും.

`model` എന്നത് `gpt-4o-mini` ആയി നിർവചിച്ചിരിക്കുന്നു. GitHub Models മാർക്കറ്റ്പ്ലേസിൽ ലഭ്യമായ മറ്റൊരു മോഡലിലേക്ക് മാറ്റി വ്യത്യസ്ത ഫലങ്ങൾ കാണാൻ ശ്രമിക്കുക.

ഒരു ലളിതമായ പരീക്ഷണമായി, നാം ഒരു ലളിതമായ പ്രോംപ്റ്റ് പ്രവർത്തിപ്പിക്കും - `What is the capital of France`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## ഫംഗ്ഷനുകൾ നിർവചിക്കുന്നത്

ഈ ഉദാഹരണത്തിൽ, ലഭ്യമായ അവധിക്കാല ഗമ്യസ്ഥലങ്ങളുടെ പട്ടികയും അവയുടെ ലഭ്യതയും ഉള്ള ഒരു ഫംഗ്ഷൻ എന്ന ടൂളിലേക്ക് ഏജന്റിന് ആക്സസ് നൽകും.

ഇത് ഒരു യാത്രാ ഏജന്റിന്, ഉദാഹരണത്തിന്, ഒരു യാത്രാ ഡാറ്റാബേസിലേക്ക് ആക്സസ് ഉള്ള ഒരു സാഹചര്യമായി ചിന്തിക്കാം.

ഈ സാമ്പിള്‍ പരിശോധിക്കുമ്പോള്‍, ഏജന്റ് വിളിക്കാവുന്ന പുതിയ ഫംഗ്ഷനുകളും ടൂളുകളും നിർവചിക്കാൻ സ്വതന്ത്രമായി ശ്രമിക്കാം.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## ഫംഗ്ഷൻ ടൂൾ നിർവചിക്കുന്നത്  
`vacation_destinations`നെ `FunctionTool` ആയി ഏജന്റ് ഉപയോഗിക്കാൻ, അതിനെ ഒരു ഫംഗ്ഷൻ ടൂളായി നിർവചിക്കേണ്ടതുണ്ട്.  

ഉപയോക്താവ് അഭ്യർത്ഥിച്ചിരിക്കുന്ന പ്രവർത്തനവുമായി ബന്ധപ്പെട്ട് ഏജന്റിന് ആ ടൂൾ എന്തിനാണ് ഉപയോഗിക്കുന്നത് എന്ന് തിരിച്ചറിയാൻ സഹായകരമായ ഒരു വിവരണം നമുക്ക് നൽകാം.  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## ഏജന്റ് നിർവചിക്കുന്നത്

ഇപ്പോൾ താഴെ കൊടുത്തിരിക്കുന്ന കോഡിൽ ഏജന്റ് സൃഷ്ടിക്കാം. ഉപയോക്താക്കൾക്ക് അവധിക്കാല ഗമ്യസ്ഥലങ്ങൾ കണ്ടെത്തുന്നതിൽ സഹായിക്കുന്നതിന് ഏജന്റിന് നിർദ്ദേശങ്ങൾ നൽകാൻ `system_message` നിർവചിക്കുന്നു.

കൂടാതെ, `reflect_on_tool_use` പാരാമീറ്റർ true ആയി സജ്ജമാക്കുന്നു. ഇത് ടൂൾ കോൾയുടെ പ്രതികരണം എടുക്കാനും സ്വാഭാവിക ഭാഷയിൽ പ്രതികരണം അയയ്ക്കാനും LLM ഉപയോഗിക്കാൻ അനുവദിക്കുന്നു.

പാരാമീറ്റർ false ആയി സജ്ജമാക്കി വ്യത്യാസം കാണാൻ നിങ്ങൾക്ക് കഴിയും.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## ഏജന്റ് പ്രവർത്തിപ്പിക്കൽ

ഇപ്പോൾ ടോക്യോയിലേക്ക് യാത്ര പോകാൻ ആവശ്യപ്പെട്ടുകൊണ്ടുള്ള പ്രാരംഭ ഉപയോക്തൃ സന്ദേശത്തോടെ ഏജന്റ് പ്രവർത്തിപ്പിക്കാം.

നിങ്ങൾ ഈ നഗര ലക്ഷ്യം മാറ്റി ഏജന്റ് നഗരത്തിന്റെ ലഭ്യതയെ എങ്ങനെ പ്രതികരിക്കുന്നു എന്ന് കാണാൻ കഴിയും.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസത്യവാദം**:  
ഈ രേഖ AI വിവർത്തന സേവനമായ [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. കൃത്യതയ്ക്കായി ഞങ്ങൾ ശ്രമിക്കുന്നുവെങ്കിലും, ഓട്ടോമേറ്റഡ് വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റായ വിവരങ്ങൾ ഉണ്ടാകാൻ സാധ്യതയുണ്ട്. അതിന്റെ സ്വാഭാവിക ഭാഷയിലുള്ള മൗലിക രേഖ പ്രാമാണികമായ ഉറവിടമായി കണക്കാക്കണം. നിർണായകമായ വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
